# Problema:
En una partido de tenis, dos tenistas disputan un punto que de partido y nosotros apoyamos al segundo tenista ¿Cuáles son las jugadas que le darían la victoria al segundo tenista?

# Representación del problema:
Nuestras variables son las posiciones de ambos tenistas (t1 y t2) y la posición de la pelota (p). En la cancha existen cuatro posiciones representadas en binario (00...11) que pueden tomar t1, t2 y p, tal que t1 y p no deben tener la misma posición. Finalmente, cada caso se representa como un número binario de 6 bits donde los últimos dos bit representan a la posición t1, los siguientes bits representan la posición de t2 y los primeros 2 bits la posición de la pelota.

* 00 -> posición dcha
* 01 -> posición ctl
* 10 -> posición izq
* 11 -> posición net

<img src="tenis_shot.png">

## Ejemplos de representaciones válidas:
* 001101 => t1 = 00, t2 = 11, p = 01
* 111100 => t1 = 11, t2 = 11, p = 00
* 010000 => t1 = 01, t2 = 00, p = 00

# Restricciones:
Para que el segundo tenista gane el punto, debe cumplirse una de las condiciones siguientes:
1. t2 y p están las posiciones 00, 01 o 10, pero en posiciones no consecutivas.
2. t1 está en la posición 01 y p en la posición 11
3. t1 está en la posición 11, t2 está en la posición 00 y p en 10 o t2 está en la posición 10 y p en 00

# Implementación

In [33]:
# Importar paquetes necesarios
import random

In [34]:
# Definición de funciones de ayuda

def int_to_bin_nbits(i,n):
    """Conversión de base 10 a base 2 pero con resultado de n cantidad de bits"""
    b = bin(i)[2:]
    if len(b) < n:
        b = '0' * (n - len(b)) + b
    return b


In [35]:
# Definición de funciones de generación de casos

def create_case():
    """Crea un caso para el problema"""
    positions = list(range(0, 4))
    t2 = random.choice(positions)
    t1 = random.choice(positions)
    del positions[positions.index(t1)]
    p = random.choice(positions)
    return "".join([int_to_bin_nbits(p, 2) for p in (t1, t2, p)])


def verify_solution(case):
    """Comprueba que el caso sea solución"""
    t1 = int(f"0b{case[:2]}", 2)
    t2 = int(f"0b{case[2:4]}", 2)
    p = int(f"0b{case[-2:]}", 2)

    cond1 = t1 <= 2 and abs(t1 - p) == 2
    cond2 = t1 == 1 and p == 3
    cond3_var1 = t2 == 0 and p == 2
    cond3_var2 = t2 == 2 and p == 0
    cond3 = t1 == 3 and (cond3_var1 or cond3_var2)

    return cond1 or cond2 or cond3


In [36]:
# Definición de métodos de cruza
cross_record = {}

def crossover(parent_1, parent_2):
    """
        Dado dos situaciones padres hace un cruza que devuelve dos situaciones hijas.

        El método de cruza es romper las representaciones padres en un mismo punto y
        generar representaciones hijas a partir de las partes de los padres.
    """
    slice_index = random.randint(1, len(parent_1) - 2)
    child_1 = parent_1[: slice_index] + parent_2[slice_index:]
    child_2 = parent_2[: slice_index] + parent_1[slice_index:]

    # Datos de puntos de corte de las cruzas
    if str(slice_index) not in cross_record:
        cross_record[str(slice_index)] = 0
    cross_record[str(slice_index)] += 1

    return child_1, child_2

In [37]:
# Definición de métodos de mutación

mut_record = {}

def mutation(case, rate):
    """Dado un porcentaje de posibilidad de mutación, la mutación consiste en cambiar un bit del caso"""
    if random.random() > rate:
        return case

    mutation_at = random.randint(0, len(case) - 1)
    prefix = "" if mutation_at == 0 else case[: mutation_at - 1]
    gen = '0' if case[mutation_at] == '1' else '1'
    suffix = case[mutation_at:]

    # Datos de bits de las mutaciones
    if str(mutation_at) not in mut_record:
        mut_record[str(mutation_at)] = 0
    mut_record[str(mutation_at)] += 1

    return  prefix + gen + suffix


# Solución del problema

In [38]:
# Variables de la solución

solutions = []
iterations = 100000
mutation_rate = .5

In [39]:
# Generar soluciones iniciales

while len(solutions) < 2:
    case = create_case()
    if verify_solution(case):
        solutions.append(case)

parent_1, parent_2 = list(solutions)
print(parent_1, parent_2)

000010 011111


In [40]:
# Encontrando más soluciones

for i in range(iterations):
    parent_1, parent_2 = crossover(parent_1, parent_2)
    for child in (parent_1, parent_2):
        mutation(child, mutation_rate)
        if verify_solution(child) and child not in solutions:
            solutions.append(child)

print('final parents:', parent_1, parent_2)
print(f'solutions: {solutions}')
print(f'total crossovers: {sum(cross_record[k] for k in cross_record)}', cross_record)
print(f'total mutations: {sum(mut_record[k] for k in mut_record)}', mut_record)

final parents: 001010 010111
solutions: ['000010', '011111', '000110', '011011', '001010', '010111', '001110', '010011']
total crossovers: 100000 {'3': 25073, '4': 25036, '1': 25006, '2': 24885}
total mutations: 100313 {'0': 16683, '1': 16730, '2': 16936, '4': 16526, '5': 16836, '3': 16602}
